***Final Project: Scheduling *** </br>
The program takes in time objects and compares against calendar events from the google calendar API to check for availability</br>
Calendar credentials and events are from jackline.w.gathoni.24@dartmouth.edu </br>
Dartmouth College, LING48, Spring 2023  </br>
Jackline Gathoni (jackline.w.gathoni.24@dartmouth.edu) </br>
Dahlia Igiraneza (dahlia.igiraneza.24@dartmouth.edu)  </br>
Paige Nakai (paige.m.nakai.24@dartmouth.edu)  </br>
Getting calendar credentials code is sourced from: https://developers.google.com/calendar/api/quickstart/python </br>

In [1]:
!pip install pytz
!pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.4/11.4 MB 114.0 MB/s eta 0:00:00
  Attempting uninstall: google-api-python-client
    Found existing installation: google-api-python-client 2.84.0
    Uninstalling google-api-python-client-2.84.0:
      Successfully uninstalled google-api-python-client-2.84.0


In [ ]:
from __future__ import print_function

import datetime
import os.path
import pytz

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from zoneinfo import ZoneInfo


# If modifying these scopes, delete the file token.json.
SCOPES = ['https://www.googleapis.com/auth/calendar.readonly']


def getEvents():
    """Shows basic usage of the Google Calendar API.
    Prints the start and name of the next 10 events on the user's calendar.
    """
    creds = None
    # The file token.json stores the user's access and refresh tokens, and is
    # created automatically when the authorization flow completes for the first
    # time.
    if os.path.exists('calendar_token.json'):
        creds = Credentials.from_authorized_user_file('calendar_token.json', SCOPES)
    # If there are no (valid) credentials available, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                './client_secret_241823786615-5oh4q20ihegr14g6bpst89ke47v65ikf.apps.googleusercontent.com.json', SCOPES)
            creds = flow.run_local_server(port=0)
        # Save the credentials for the next run
        with open('calendar_token.json', 'w') as token:
            token.write(creds.to_json())

    try:
        service = build('calendar', 'v3', credentials=creds)

        # Call the Calendar API
        now = datetime.datetime.utcnow().isoformat() + 'Z'  # 'Z' indicates UTC time
        print('Getting the upcoming 10 events...')
        events_result = service.events().list(calendarId='primary', timeMin=now,
                                              maxResults=10, singleEvents=True,
                                              orderBy='startTime').execute()
        events = events_result.get('items', [])

        if not events:
            print('No upcoming events found.')
            return

      
    except HttpError as error:
        print('An error occurred: %s' % error)
    return events


def parseDate(rawDate):
    #Transform the datetime given by the API to a python datetime object.
    return datetime.datetime.strptime(rawDate[:-6]+ rawDate[-6:].replace(":",""), '%Y-%m-%dT%H:%M:%S%z')


def checkAvailability(start_time, end_time):
        events = getEvents()

        for event in events:

            start_date= datetime.datetime.combine(datetime.datetime.today(),start_time )
            end_date= datetime.datetime.combine(datetime.datetime.today(),end_time )

            eventStart = parseDate(event['start'].get('dateTime', event['start'].get('date'))) 
            eventEnd = parseDate(event['end'].get('dateTime', event['end'].get('date')))
            
                # Check for an overlap between the event and the desired interval
            if eventStart < end_date and eventEnd> start_date:
                print('The desired interval overlaps with event:', event['summary'])
                return False
            
        return True